In [0]:
import pyspark.sql.functions as F
from datetime import date, timedelta

dbutils.widgets.text("RUN_DATE", "")
RUN_DATE = dbutils.widgets.get("RUN_DATE")

if not RUN_DATE:
    # Best default for historical datasets: latest available date in Bronze
    RUN_DATE = (
        spark.table("workspace.default.bronze_events")
             .agg(F.max("event_date").alias("d"))
             .collect()[0]["d"]
    )
    RUN_DATE = str(RUN_DATE)

print("✅ RUN_DATE =", RUN_DATE)


In [0]:
from pyspark.sql import functions as F

SILVER_TABLE = "workspace.default.silver_events"
GOLD_PRODUCT_TABLE = "workspace.default.gold_product_funnel_daily"

# Use the latest available day in Silver (recommended)
RUN_DATE = spark.table(SILVER_TABLE).agg(F.max("event_date").alias("d")).collect()[0]["d"]

print("Using RUN_DATE =", RUN_DATE)

silver_day = (
    spark.table(SILVER_TABLE)
    .where(F.col("event_date") == F.lit(RUN_DATE))
)

print("✅ Silver rows for RUN_DATE:", silver_day.count())
display(silver_day.select("event_time","event_type","product_id","category_code","brand","price").limit(10))


In [0]:
group_cols = ["event_date", "product_id", "category_code", "brand"]

gold_product = (
    silver_day
    .groupBy(*group_cols)
    .agg(
        F.sum(F.when(F.col("event_type") == "view", 1).otherwise(0)).cast("long").alias("view_events"),
        F.sum(F.when(F.col("event_type") == "cart", 1).otherwise(0)).cast("long").alias("cart_events"),
        F.sum(F.when(F.col("event_type") == "remove_from_cart", 1).otherwise(0)).cast("long").alias("remove_from_cart_events"),
        F.sum(F.when(F.col("event_type") == "purchase", 1).otherwise(0)).cast("long").alias("purchase_events"),

        # Revenue only for purchase events (price can be null for non-purchase)
        F.sum(F.when(F.col("event_type") == "purchase", F.col("price").cast("double")).otherwise(F.lit(0.0))).alias("purchase_revenue")
    )
    .withColumn("view_to_cart_rate",
                F.when(F.col("view_events") > 0, F.col("cart_events") / F.col("view_events")).otherwise(F.lit(None).cast("double")))
    .withColumn("cart_to_purchase_rate",
                F.when(F.col("cart_events") > 0, F.col("purchase_events") / F.col("cart_events")).otherwise(F.lit(None).cast("double")))
    .withColumn("view_to_purchase_rate",
                F.when(F.col("view_events") > 0, F.col("purchase_events") / F.col("view_events")).otherwise(F.lit(None).cast("double")))
    .withColumn("run_date", F.lit(RUN_DATE).cast("date"))
)

print("✅ Aggregated product-funnel rows:", gold_product.count())
display(gold_product.orderBy(F.col("purchase_events").desc()).limit(20))
gold_product.printSchema()


In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {GOLD_PRODUCT_TABLE} (
  event_date date,
  product_id long,
  category_code string,
  brand string,
  view_events long,
  cart_events long,
  remove_from_cart_events long,
  purchase_events long,
  purchase_revenue double,
  view_to_cart_rate double,
  cart_to_purchase_rate double,
  view_to_purchase_rate double,
  run_date date
)
USING DELTA
PARTITIONED BY (event_date)
""")

print(f"✅ Table ready: {GOLD_PRODUCT_TABLE}")


In [0]:
(
    gold_product.write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"event_date = DATE('{RUN_DATE}')")
    .saveAsTable(GOLD_PRODUCT_TABLE)
)

print(f"✅ Gold product funnel written for {RUN_DATE}")


In [0]:
gold_prod_check = spark.table(GOLD_PRODUCT_TABLE).where(F.col("event_date") == F.lit(RUN_DATE))

print("✅ Rows in GOLD_PRODUCT_TABLE for RUN_DATE:", gold_prod_check.count())

# Quick totals across products should match Step 1 daily totals
totals = gold_prod_check.agg(
    F.sum("view_events").alias("total_views"),
    F.sum("cart_events").alias("total_carts"),
    F.sum("remove_from_cart_events").alias("total_removes"),
    F.sum("purchase_events").alias("total_purchases"),
    F.sum("purchase_revenue").alias("total_revenue")
)

display(totals)

display(
    gold_prod_check.orderBy(F.col("purchase_events").desc(), F.col("view_events").desc()).limit(20)
)

gold_prod_check.printSchema()


In [0]:
GOLD_PRODUCT_TABLE = "workspace.default.gold_product_funnel_daily"

spark.sql(f"DROP TABLE IF EXISTS {GOLD_PRODUCT_TABLE}")

# Create empty Delta table with the exact schema of gold_product
gold_product.limit(0).write.format("delta").saveAsTable(GOLD_PRODUCT_TABLE)

print(f"✅ Gold product funnel table recreated: {GOLD_PRODUCT_TABLE}")


In [0]:
spark.sql(f"DROP TABLE IF EXISTS {GOLD_PRODUCT_TABLE}")

spark.sql(f"""
CREATE TABLE {GOLD_PRODUCT_TABLE} (
  event_date date,
  product_id long,
  category_code string,
  brand string,
  view_events long,
  cart_events long,
  remove_from_cart_events long,
  purchase_events long,
  purchase_revenue double,
  view_to_cart_rate double,
  cart_to_purchase_rate double,
  view_to_purchase_rate double,
  run_date date
)
USING DELTA
PARTITIONED BY (event_date)
""")

print(f"✅ Gold product funnel table ready: {GOLD_PRODUCT_TABLE}")


In [0]:
from pyspark.sql import functions as F

# Ensure run_date is a DATE
run_date_expr = f"event_date = DATE('{RUN_DATE}')"

(
  gold_product
    .write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")        # safe if schema evolves
    .option("replaceWhere", run_date_expr) # overwrite only this day
    .saveAsTable(GOLD_PRODUCT_TABLE)
)

print(f"✅ Gold product funnel written for {RUN_DATE} into {GOLD_PRODUCT_TABLE}")


In [0]:
gold_prod_check = (
  spark.table(GOLD_PRODUCT_TABLE)
       .where(F.col("event_date") == F.lit(RUN_DATE).cast("date"))
)

print("✅ Rows in GOLD_PRODUCT_TABLE for RUN_DATE:", gold_prod_check.count())

display(
  gold_prod_check
    .orderBy(F.col("purchase_events").desc(), F.col("view_events").desc())
    .limit(20)
)

gold_prod_check.printSchema()


In [0]:
from datetime import date, timedelta

dbutils.widgets.text("RUN_DATE", "")
RUN_DATE = dbutils.widgets.get("RUN_DATE")

if not RUN_DATE:
    # default = yesterday
    RUN_DATE = str(date.today() - timedelta(days=1))

print("✅ RUN_DATE =", RUN_DATE)
